In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.data import shepp_logan_phantom
from skimage.transform import resize

def load_image(size=128):
    image = shepp_logan_phantom() 
    image = resize(image, (size, size), mode='reflect', anti_aliasing=True)
    return image

def radon_transform(image, angles, num_detectors, detector_span):
    sinogram = np.zeros((len(angles), num_detectors))
    center = image.shape[0] // 2
    detector_positions = np.linspace(-detector_span / 2, detector_span / 2, num_detectors)
    
    for i, angle in enumerate(angles):
        emitter_x = center + np.cos(np.radians(angle)) * center
        emitter_y = center + np.sin(np.radians(angle)) * center
        for j, detector_offset in enumerate(detector_positions):
            detector_x = center + np.cos(np.radians(angle + 180)) * center + np.sin(np.radians(angle)) * detector_offset
            detector_y = center + np.sin(np.radians(angle + 180)) * center - np.cos(np.radians(angle)) * detector_offset
            sinogram[i, j] = integrate_ray(image, emitter_x, emitter_y, detector_x, detector_y)
    return sinogram

def integrate_ray(image, x1, y1, x2, y2):
    x, y = np.linspace(x1, x2, 1000), np.linspace(y1, y2, 1000)
    x, y = np.clip(x.astype(int), 0, image.shape[0] - 1), np.clip(y.astype(int), 0, image.shape[1] - 1)
    return np.sum(image[x, y])

def inverse_radon_transform(sinogram, angles, image_size):
    reconstructed_image = np.zeros((image_size, image_size))
    center = image_size // 2
    for i, angle in enumerate(angles):
        for j, value in enumerate(sinogram[i]):
            detector_x = center + np.cos(np.radians(angle + 180)) * center
            detector_y = center + np.sin(np.radians(angle + 180)) * center
            add_ray(reconstructed_image, detector_x, detector_y, value)
    return reconstructed_image

def add_ray(image, x, y, value):
    pass 

def visualize(image, sinogram, reconstructed_image):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title("Input Image")
    axes[1].imshow(sinogram, cmap='gray', aspect='auto')
    axes[1].set_title("Sinogram")
    axes[2].imshow(reconstructed_image, cmap='gray')
    axes[2].set_title("Reconstructed Image")
    plt.show()

def main():
    image = load_image()
    angles = np.linspace(0, 180, 180, endpoint=False)
    num_detectors = 128
    detector_span = 128
    sinogram = radon_transform(image, angles, num_detectors, detector_span)
    reconstructed_image = inverse_radon_transform(sinogram, angles, image.shape[0])
    visualize(image, sinogram, reconstructed_image)

main()